# CSE 493 G1 Final Project
Make sure to change runtime to GPU

In [ ]:
# connected to drive
from google.colab import drive
dir = '/content/drive'
drive.mount(dir)

import os
data_dir = os.path.join(dir, "MyDrive", "")

Mounted at /content/drive


Pip cell: do not rerun

In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install wandb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


# Data Preprocessing

In [ ]:
# get staff posts, and all posts
import json

def get_text_content(post, docs, is_admin=lambda _: True):
    if is_admin(post):
        docs.append(post['document'])
    for comment in post['comments']:
        get_text_content(comment, docs, is_admin)

# get only staff posts
def get_staff_posts(threads):
    staff_posts = []

    for t in threads:
        users = t['users']
        thread = t['thread']

        def is_admin(post):
            id = post['user_id']
            for user in users:
                if id == user['id'] and user['course_role'] == 'admin' or user['role'] == 'admin':
                    return True
            return False

        get_text_content(thread, staff_posts, is_admin)

    return staff_posts

def get_posts(threads):
    posts = []
    for t in threads:
        thread = t['thread']
        get_text_content(thread, posts)
    return posts

with open('/content/drive/MyDrive/project-data/DL_493G1.json', 'r') as f:
    threads = json.loads(f.read())

posts = get_staff_posts(threads)

In [ ]:
# convert into dataset
from datasets import Dataset

posts = list(map(lambda x: {'text': x}, posts))
dataset = Dataset.from_list(posts)
dataset

Dataset({
    features: ['text'],
    num_rows: 39
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_checkpoint = 'openai-community/gpt2'
# model_checkpoint = 'filipealmeida/Mistral-7B-v0.1-sharded'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
tokenized_dataset

Map (num_proc=4):   0%|          | 0/39 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 39
})

In [ ]:
from datasets import Dataset, concatenate_datasets, DatasetDict

tokenized_dataset_split = tokenized_dataset.train_test_split(test_size=0.2, shuffle=True)
tokenized_dataset_split['val'] = tokenized_dataset_split.pop('test')

# Uncomment this if you want a train/test/val split
'''
test_and_val = tokenized_dataset_split['test'].train_test_split(test_size=0.5, shuffle=True)
all_data = DatasetDict({
  'train': tokenized_dataset_split['train'],
  'test': test_and_val['test'],
  'val': test_and_val['train']
})
all_data
'''

tokenized_dataset_split

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 31
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8
    })
})

In [ ]:
block_size = 512 # 512

def group_texts(examples):
    # Concatenate all texts in one batch. For example, [[1,2], [3,4]] -> [1,2,3,4]
    # Keys are: input_ids, attention_mask
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset_split.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)
lm_dataset

Map (num_proc=4):   0%|          | 0/31 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/8 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 0
    })
})

# Fine-tuning

In [ ]:
# more imports
import os, torch, platform, warnings
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import wandb

In [ ]:
# Load base model(gpt2)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# model.config.use_cache = False # make this true during inference
# model.config.pretraining_tp = 1
# model.gradient_checkpointing_enable()
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
'''
# Hyperparams oh boy

# Normal trainer
# other parameters to consider: optim, adam_stuff, lr_scheduler_type, warmup_ratio
'''
import gc
gc.collect()
torch.cuda.empty_cache()

model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    f"{model_name}-sphishing",
    evaluation_strategy = "epoch",
    num_train_epochs=30,
    learning_rate=1e-5,
    weight_decay=0.9,
    report_to="wandb",
    lr_scheduler_type="cosine",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["val"],
)

In [ ]:
"""
Tuning notes:


param settings tried UGRAD:
  lr: 1e-3, 1e-5 BAD
  e: 5,  lr: 1e-4, decay: 0 (2.74 val loss, slight overfit at end)
  e: 5,  lr: 1e-4, decay: 0.01 (2.74 val loss, slight over)
  *e: 5,  lr: 1e-4, decay: 0.5 (2.73 val loss, no over, saved as ok_UGRAD)
  e: 5,  lr: 5e-5, decay: 0 (2.85 val, slow, no overfit)
  e: 10, lr: 5e-5, decay: 0.05 (2.76 val, slow, no overfit)
  *e: 5,  lr: 2e-4, decay: 0.9 (2.69 val loss, overfit)

param settings tried CAREER:
  e: 5,  lr: 2e-5, decay: 0.01 (3.13 val)
  e: 5,  lr: 2e-4, decay: 0.01 (overfit, 3.08 val)
  e: 5,  lr: 1e-4, decay: 0.5 (2.99 val)
  e: 5,  lr: 2e-4, decay: 0.9 (3.01 val)
  e: 5,  lr: 1e-5, decay: 0.01 (3.19 val)
  e: 10, lr: 1e-5, decay: 0.01 (3.17 val)
  *e: 20, lr: 1e-5, decay: 0.01, no scheduler (3.09 val, saved as ok_CAREER)

param settings tried 493:

"""

'\nTuning notes:\n\n\nparam settings tried UGRAD:\n  lr: 1e-3, 1e-5 BAD\n  e: 5,  lr: 1e-4, decay: 0 (2.74 val loss, slight overfit at end)\n  e: 5,  lr: 1e-4, decay: 0.01 (2.74 val loss, slight over)\n  *e: 5,  lr: 1e-4, decay: 0.5 (2.73 val loss, no over, saved as ok_UGRAD)\n  e: 5,  lr: 5e-5, decay: 0 (2.85 val, slow, no overfit)\n  e: 10, lr: 5e-5, decay: 0.05 (2.76 val, slow, no overfit)\n  *e: 5,  lr: 2e-4, decay: 0.9 (2.69 val loss, overfit)\n\nparam settings tried CAREER:\n  e: 5,  lr: 2e-5, decay: 0.01 (3.16 val)\n  e: 5,  lr: 2e-4, decay: 0.01 (overfit, 3.08 val)\n  e: 5,  lr: 1e-4, decay: 0.5 (overfit, 3.36)\n\n'

In [ ]:
# train
trainer.train()

model.config.use_cache = True
model.eval()
eval_results = trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,No log,3.767177
2,No log,3.761048
3,No log,3.755820
4,No log,3.749516
5,No log,3.744103
6,No log,3.740183
7,No log,3.736799
8,No log,3.734459
9,No log,3.732896
10,2.662400,3.732056


In [ ]:
# DONT OVERWRITE IF WANT NEW MODEL
save_model_name = 'ok_493G1_staff_only_model'
model_dir = '/content/drive/MyDrive/project-data/'

In [ ]:
trainer.save_model(model_dir + save_model_name) # change name to save different model!!!

import math
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 42.25


# Decoding

In [ ]:
"""
# Decoding notes ok_UGRAD

Greedy: BAD, repetitive

Contrastive: starts repeating after a while, higher top_k (4, 5, 6) or higher pen_alpha (0.8 good) reduce repetition

Multinomial: no repetition but kinda nonsensical and no continuity

Beam: meh, very short messages, kinda repeats

Beam+multinomial: ig better than beam, still short (could be issue with prompt)

diverse beam: repeats?

sample, topk50, topp 0.95: diverse, continuinity bad
  base GPT2: not UW/allen specific

"""

In [ ]:
# ONLY RUN FOR LOADING PRETRAINED MODEL
model = AutoModelForCausalLM.from_pretrained(model_dir + 'ok_493G1_staff_only_model')

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)
input = 'Hello CSE students (undergrads and grad students)! We have an important announcement for the upcoming Spring 2024 quarter. Visit this link to'
model_inputs = tokenizer(input, return_tensors='pt').to(torch_device)
model_outputs = model.generate(**model_inputs,
                        max_new_tokens=100,
                        do_sample=True,
                        top_k=50,
                        top_p=0.95,
                        num_return_sequences=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


cpu


In [ ]:
print("Output:\n" + 100 * '-')
for i, model_output in enumerate(model_outputs):
  print("{}: {}".format(i, tokenizer.decode(model_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: Hello CSE students (undergrads and grad students)! We have an important announcement for the upcoming Spring 2024 quarter. Visit this link to learn more and see the details. See you in Spring. See you in Spring!

Please follow the main announcement on the website to learn more about your new promotion!
1: Hello CSE students (undergrads and grad students)! We have an important announcement for the upcoming Spring 2024 quarter. Visit this link to check the news. We want to give everyone some time to learn about the project!

The project is now open for developers to create a private mailing list (https://github.com/Raleigh-Founded-PBS-Community-Project-Releases/tree/master/master) that features bug reporting and bug-fixing information. Open the mailing list to ask questions, submit pull requests, or have your project discussed with staff.

For all the latest information on
2: H

#### decoding the custom-loss model

In [ ]:
assistant_model = AutoModelForCausalLM.from_pretrained('distilbert/distilgpt2')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_dir='/content/drive/MyDrive/project-data/'

model = AutoModelForCausalLM.from_pretrained(model_dir + 'sphishing_model')
tokenizer = AutoTokenizer.from_pretrained(model_dir + 'sphishing_tokenizer')

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
input = 'Hello CSE students (undergrads and grad students)! We have an important announcement for the upcoming Spring 2024 quarter. Visit this link to'
model_inputs = tokenizer(input, return_tensors='pt').to(torch_device)
'''
model_outputs = model.generate(**model_inputs,
                        max_new_tokens=200,
                        assistant_model=assistant_model)
'''

model_outputs = model.generate(**model_inputs,
                        max_new_tokens=200,
                        do_sample=True,
                        top_k=80,
                        top_p=0.95,
                        num_return_sequences=5)

In [ ]:
print("Output:\n" + 100 * '-')
for i, model_output in enumerate(model_outputs):
  print("{}: {}".format(i, tokenizer.decode(model_output, skip_special_tokens=True)))
